# import

In [1]:
import json
import os
import re
from pprint import pprint

from soynlp.normalizer import repeat_normalize

In [2]:
def dump_jsonl(data, output_path, append=False):
    mode = "a+" if append else "w"
    with open(output_path, mode, encoding="utf-8") as f:
        for line in data:
            json_record = json.dumps(line, ensure_ascii=False)
            f.write(json_record + "\n")


def load_jsonl(input_path) -> list:
    data = []
    with open(input_path, "r", encoding="utf-8") as f:
        for line in f:
            data.append(json.loads(line.rstrip("\n|\r")))
    return data

In [3]:
# '/home/jh/documents/datasets/git_category.json'
os.chdir("/home/jh/documents")

vocab_size = 24000

# load data

In [4]:
def processing(text):
    #     text = re.sub(r' +', r' ', text.strip())
    #     text = re.sub(r'(.{8,}?)\1+', r'\1', text)
    #     text = re.sub(r'[^ ㄱ-ㅎㅏ-ㅣ가-힣A-Za-z0-9~!@#$%\^\&*\(\)_\+\-=\[\]{},\./<>\?]', r'', text)
    #     text = re.sub(r'http.+', r'<url>', text)

    return text.strip()

In [5]:
data = load_jsonl("datasets/git_category.json")

In [6]:
data[0]["content"]

'서울박정호 SKT 대표이사가 25일 서울 중구 을지로 SKT타워에서 열린 SK텔레콤 제37기 정기 주주총회에 참석하고 있다SK텔레콤이 주주가치 제고를 위해 한국거래소에 자사주 869만주 소각을 반영한 변경상장을 완료했다고 14일 밝혔다 발행주식 총수의약 2조6000억원규모다 이로써 전날 기준 SK텔레콤의 발행주식 총수는 기존 8075만주에서 7206만주로 줄어들었다앞서 SK텔레콤은 지난 4일 올해 인적분할에 앞서 기업가치주주가치 제고를 위해 기존 자사주를 사실상 전량 소각한다고 발표한 바 있다 이번 자사주 소각으로 발행주식 총수가 감소하면서 기존 주주들의 지분율이 모두 상승했다SK텔레콤은 분할 후 기업가치가 올라갈 것으로 전망하는 증권업계 전반의 시각을 감안하면 기업 펀더멘털 변동없이 주식 수만 줄어든 상황이라며 자사주 소각 전보다 주식가치 상승 여력이 더 커졌다고 분석했다SK텔레콤은 발행주식 총수가 줄었음에도 전날 시가총액이 약 22조5000억원을 기록해 주주총회에서 기업구조인적분할 개편을 공식화했던 지난 3월25일 시가총액약 20조5000억원보다 10가량 늘었다고 설명했다 주가도 연초 대비 30 이상 상승했다고 덧붙였다한편 SK텔레콤은 상반기 내 이사회 의결을 거쳐 오는 10월 주주총회 11월 인적분할 법인의 재상장을 통해 인적분할 절차를 끝낼 계획이다'

In [7]:
def gen():
    for row in data:
        yield row["content"]

In [8]:
user_defined_symbols = [
    "<pad>",
    "<unk>",
    "<cls>",
    "<sep>",
    "<mask>",
    "<bos>",
    "<eos>",
]

unused_token_num = 100
unused_list = [f"<unused{i}>" for i in range(unused_token_num)]
whole_user_defined_symbols = user_defined_symbols + unused_list

pprint(user_defined_symbols)

['<pad>', '<unk>', '<cls>', '<sep>', '<mask>', '<bos>', '<eos>']


# train

In [9]:
from tokenizers import Tokenizer
from tokenizers.models import BPE

gpt_tokenizer = Tokenizer(BPE(unk_token="<unk>"))

In [10]:
from tokenizers import normalizers
from tokenizers.normalizers import NFKC, BertNormalizer


n1 = NFKC()
n2 = BertNormalizer(
    clean_text=False,
    handle_chinese_chars=False,
    strip_accents=False,
    lowercase=False,
)

gpt_tokenizer.normalizer = normalizers.Sequence([n1, n2])

In [11]:
from tokenizers import pre_tokenizers
from tokenizers.pre_tokenizers import Metaspace

gpt_tokenizer.pre_tokenizer = pre_tokenizers.Sequence(
    [Metaspace(replacement="_", add_prefix_space=True)]
)

In [12]:
# post_processing pass

In [12]:
from tokenizers.trainers import BpeTrainer

trainer = BpeTrainer(
    vocab_size=vocab_size,
    special_tokens=whole_user_defined_symbols,
)
gpt_tokenizer.train_from_iterator(gen(), trainer)

In [13]:
output = gpt_tokenizer.encode("테스트용인데 잘 되는거같아?")
print(output.ids)

gpt_tokenizer.decode(output.ids)

[6733, 1667, 3881, 3146, 3703, 233, 216, 1533, 1]


'_테스트 용 인데 _잘 _되는 거 같 아'

In [14]:
from tokenizers import decoders

gpt_tokenizer.decoder = decoders.BPEDecoder(suffix="_")
gpt_tokenizer.decode(output.ids)

'테스트용인데 잘 되는거같아'

# convert transformers tokenizer and save

In [15]:
from transformers import GPT2TokenizerFast


fast_tokenizer = GPT2TokenizerFast(tokenizer_object=gpt_tokenizer)

/home/jh/.pyenv/versions/3.8.15/envs/jupyter/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
fast_tokenizer.pad_token = "<pad>"
fast_tokenizer.unk_token = "<unk>"
fast_tokenizer.cls_token = "<cls>"
fast_tokenizer.sep_token = "<sep>"
fast_tokenizer.mask_token = "<mask>"
fast_tokenizer.bos_token = "<bos>"
fast_tokenizer.eos_token = "<eos>"

special_tokens_dict = {"additional_special_tokens": user_defined_symbols}
fast_tokenizer.add_special_tokens(special_tokens_dict)

0

In [17]:
e = fast_tokenizer.encode("<user1>'테스트용'으로 \"잘\" 되는지 보고이따<sep>")
print(e)
print(fast_tokenizer.decode(e))

[143, 1, 10283, 3659, 108, 1, 1, 20022, 1667, 1, 2560, 143, 1, 1758, 1, 3703, 1873, 3170, 1734, 751, 3]


2022-11-29 17:29:49.731964: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 17:29:49.849025: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-29 17:29:50.393589: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-29 17:29:50.393654: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

<unk>user1<unk><unk>테스트용<unk>으로 <unk>잘<unk> 되는지 보고이따<sep>


In [18]:
fast_tokenizer.save_pretrained("etc/DialogBPE")

('etc/DialogBPE/tokenizer_config.json',
 'etc/DialogBPE/special_tokens_map.json',
 'etc/DialogBPE/vocab.json',
 'etc/DialogBPE/merges.txt',
 'etc/DialogBPE/added_tokens.json',
 'etc/DialogBPE/tokenizer.json')